In [ ]:
# Давайте установим необходимую библиотеку PyTorch
!pip install -q torch==1.6.0
# и импортируем ее
import torch

In [ ]:
# Импорт необходимых модулей 
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
TEXT_COLOR = 'black'

matplotlib.rcParams['figure.figsize'] = (15, 10)
matplotlib.rcParams['text.color'] = 'black'
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['axes.labelcolor'] = TEXT_COLOR
matplotlib.rcParams['xtick.color'] = TEXT_COLOR
matplotlib.rcParams['ytick.color'] = TEXT_COLOR

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

# Нейросети и PyTorch

На момент этой практики мы уже познакомились с некоторыми алгоритмами машинного обучения и даже попробовали работать с текстом. Это отличные результаты и теперь пора перейти к более узкой теме - нейросети (и обычные и глубокие). Для разработки и использования мы будем применять фреймворк PyTorch. Как всегда бывает у хорошего фреймворка, у него есть [сайт с документацией](https://pytorch.org/docs/stable/index.html).



# Понятие нейрона

Для освоения мы должны понять, из чего состоят нейросети, а состоят они из нейронов! Современной моделью нейрона является нейрон Розенблатта или *перцептрон*. Его можно отобразить в следующем виде:

<center><img src="https://docs.google.com/uc?export=download&id=1pHzbK9OnHPqbqZ1MJiIie-RGk7QJji65"/></center>

В нейроне можно выделить следующие части:
- **Входы** - это линии, по которым нейрон получает сигналы из, например, выходов других нейронов.

- **Веса** - это коэффициенты, которые показывают, насколько сигнал конкретного входа важен для нейрона. Видите, у каждого входа свой вес. По аналогии с линейной или логистической регрессией, веса - это параметры нейрона, которые меняются (обучаются) в ходе обучения. Также по аналогии с линейной регрессией здесь есть и *bias* ($b$).

- **Сумматор** - этап объединения взвешенных входов. Какого-то особого смысла не имеет, можно также провести аналогию с моделью линейной регрессии.

- **Функция активации** - вот тут и кроется самая важная часть нейрона. До данного момента все этапы имели линейный характер. Если мы сделаем нейросеть только из нейронов без функции активации, то вся нейросеть будет иметь линейный характер и ее ценность будет не больше любой линейной модели. Чаще всего функция активации - нелинейная функция. Мы уже ознакомили с одной из распространенных функций, которая используется в нейросетях - сигмоида. Помимо этого есть и другие, отобразим парочку из них:

In [ ]:
x = np.linspace(-10, 10, 100)

plt.figure(figsize=[20,5])
plt.subplot(141)
plt.plot(x, 1 / (1 + np.exp(-x)))
plt.title('Сигмоида (sigm)')
plt.grid()

plt.subplot(142)
plt.plot(x, np.tanh(x))
plt.title('Гиперболический тангенс (tanh)')
plt.grid()

plt.subplot(143)
plt.plot(x, np.maximum(x, 0))
plt.title('ReLU - Rectified Linear Unit')
plt.grid()

plt.subplot(144)
plt.plot(x, x>0)
plt.title('Пороговая функция')
plt.grid()

plt.tight_layout()
plt.show()

Вот тут мы и подошли к одному из важнейших факторов, которые отличают нейросети от остальных моделей. Нейрон в своей минималистичности (сумма взвешенных входов) с нелинейной функцией активации уже является *нелинейной* моделью. Если мы сложим из таких нейронов нейросеть, то получим **универсальный аппроксиматор**. Сложно? Проще - Какую бы хитрую вы функцию не выдумали (квадрат косинуса от корня экспоненты в степени икс), можно взять нейросеть достаточного размера (по количеству нейронов в ней) и она сможет аппроксимировать ее, то есть повторить зависимость $y=f(x)$.

## Формулы для нейрона

Теперь, когда мы разобрали, из чего нейрон состоит, можно перейти к математическому описанию - глянем на формулы.

Модели (функцию предсказания) мы описывали в виде $\hat{y} = h_W(X)$. В нашем случае $h_W(X) = g(XW)$, где $g(z)$ - функция активации. Какую функцию активации выбрать? Над этим вопросом бьются ученые со всего мира, какая же всё-таки лучше, так как выбор влияет на работу сети очень сильно.

> Тип функции активации является гиперпараметром нейрона и нейросети.

Не забывайте, что в нейроне есть *bias* ($b$). Суть его точно такая же, как в линейной регрессии. Чтобы его описать, мы точно также добавляем в $X$ колонку единиц и $W$ в нейроне - вектор с размером $(M+1)$ ($M$ - количество входов) нейрона.

> Мы в этой части не будем писать код, потому что вы уже делали подобные вещи, но крайне рекомендуем почитать материал, так как на нем строятся дальнейшие выводы.



## Обучение нейрона

> На этом моменте мы не заостряем внимание. Основная причина в том, что на сегодняшний день написание своей логики обучения нужно либо для исследовательских целей, либо для очень глубокого погружения, и своя реализация с нуля может иметь очень много ошибок. Тем не менее, вопросы обучения очень широки, поэтому крайне рекомендуем ознакомиться с различными методами и подходами в дополнительных материалах.

Учится нейрон очень похожим образом с другими моделями по методу градиентного спуска. Главная задача в обучении - найти такие значения весов, чтобы выход нейрона максимально соответсвовал данным, на которых обучается (не забываем, что здесь не учитывается шум в данных - слишком сильное соответсвие шуму может привести к переобучению, что очень плохо).

Как обычно, градиентный спуск работает по следующему принципу
$$
W := W - \alpha \frac{\partial J(W)}{\partial W}
$$

$W$ - веса нейрона, $J(W)$ - функция потерь (как обычно, выбирается в зависимости от задачи).

Для примера регрессии, возьмем функцию $MSE$ в качестве функции потерь и рассмотрим случай с одной записью (чтобы не писать громоздкую сумму и остальное):
$$
J(W) = (y-h_W(x))^2 = (y-g(x*W))^2
$$

> Тут маленький $x$, потому что мы рассматриваем конкретную запись в данных, а не весь набор.

Так вот, у нас в формуле появилась функция активации нейрона, которая имеет нелинейный характер. Что нам с ней делать? Просто расписать градиент по цепочке:
$$
\frac{\partial J(W)}{\partial W} =
\frac{\partial J(W)}{\partial g(z)} \frac{\partial g(z)}{\partial z} \frac{\partial z}{\partial W}
$$

Распишем производные, ведь мы уже находили их:
$$
\frac{\partial J(W)}{\partial g(z)} = -(y-g(z)) \\
\frac{\partial g(z)}{\partial z} = g(z)*(1-g(z)) \\
\frac{\partial z}{\partial W} = X^T
$$

Тогда можно записать полную формулу для градиента:
$$
\frac{\partial J(W)}{\partial W} = X^T*(g(XW)*(1-g(XW))*(g(XW)-y))
$$

Таким образом, нам нужно еще и для каждой функции активации знать ее градиент, но это не сложно, мы уже искали градиент сигмоиды, остальные тоже можно найти.

Дальше как обычно, обновляем веса с коэффициентом обучения $\alpha$ и идем к следующей итерации.

Все остальные шаги уже делались, так что обучение отдельного нейрона не составляет сложности! Поехали дальше!

# Нейросеть

Отлично, нейрон мы освоили, но если сказать честно, сегодня единственный нейрон нигде не применяется. Это не значит, что он бесполезен! Давайте возьмем пачку таких нейронов и выстроим в слои:

> Нейросеть (по аналогии с мозгом) - это много соединенных нейронов между собой, поэтому для большинства нейронов выходные сигналы идут на вход другим нейронам.

> Разработка нейросети на компьютере делает нейросеть **искусственой**, поэтому сигналами являются передаваемые числа (или уровень сигнала).

<center><img src="https://docs.google.com/uc?export=download&id=1T3RURXveKUqEdlljOHYqs1Aorav5uCRa"/></center>

Целых три слоя, судя по картинке, но давайте разберемся, почему же реальных слоев тут всё-таки *два* =)

- **Input layer (входной слой)** - обратим внимание, что каждый "нейрон" здесь имеет единственный вход. Выход на самом деле у каждого нейрона в слое один, просто он идет на все нейроны в следующем слое. 

> Такая форма сети называется **полносвязной** - когда мы соединяем все выходы предыдущего слоя со входами следующего.

- **Hidden layer (скрытый слой)** - он называется скрытым, потому что он не вход и не выход. Все что между - скрытые слои. Каждый нейрон в этом слое имеет столько входов, сколько нейронов у нас в предыдущем слое. В данном случае мы имеем по два входа у каждого нейрона этого слоя.

> Помним, что у нейрона несколько входов, единственный bias и единственный выход.

- **Output layer (выходной слой)** - тут особенность обратная входному, полносвязное соединение с предыдущим скрытым и по одному выходу на нейрон. Но ведь у нейрона и так всегда один выход? Да, просто дальше слоев нет, поэтому на рисунке нет столько стрелочек на выходе.

Теперь, почему тут на самом деле два слоя нейронов, а не три? Помним, что нейрон - это веса для входов и *bias*, суммирование и функция активации? Так вот у входных "нейронов" как правило нет весов и функции активации. Ну и суммировать нечего, если в входном слое один нейрон - один вход. Это схематичное отображение того, что каждый нейрон в первом скрытом слое берет все значения входов. Так просто на схемах проще показывать.

> А что является входом для нейросети? Да как и для всех моделей в ML - числовые значения признаков!

Такая структура называется **многослойной**. Для случая, когда у нас нет скрытого слоя, то остается единственный слой с весами - **однослойная структура**. Хотим сделать два скрытых? Не вопрос - делайте! =)

> Архитектура нейросети (количество слоев и количество нейронов в слое) является гиперпараметром модели.

## Функция предсказания нейросети

Переходя к функции предсказания, мы должны понять, что в нейросети каждый нейрон имеет свои веса. Давайте для примера рассмотрим скрытый слой, там мы имеем 3 нейрона, по 2 входа у каждого нейрона (так как на предыдущем "слое" у нас два нейрона). Таким образом, каждый нейрон имеет вектор размерностью $(3)$ и мы можем разместить эти вектора в матрицу, чтобы это была матрица весов скрытого слоя (на схеме $W_{ih}$ - размер $(2+1, 3)$). Аналогичным образом строим матрицу весов для выходного слоя ($W_{ho}$), 2 нейрона, 3 входа у каждого - размер $(3+1, 2)$

> Вектор весов складывается из весов для каждого входа + веса для входа bias, который всегда равен единице.

Теперь, когда мы понимаем, что отдельные нейроны в слое не рассматриваются, а мы рассматриваем весь слой и описываем его матрицей, то стоит понять еще одну важную вещь: функция активации берется для всего слоя одна. То есть, если мы приняли для конкретного слоя функцию активации *сигмоида*, то все нейроны в этом слое должны применять ее к выходам.

После таких умозаключений давайте сформируем функцию предсказания для слоев (индекс сверху):
$$
h_W^{h}(X) = g^{h}(XW) \\
h_W^{o}(X) = g^{o}(XW)
$$

Обратите внимание, это просто формулы, но они одинаковы, если мы рассматриваем каждый слой по отдельности без связи в нейросети и со своей функцией активации. Так а как будет выглядеть функция предсказания нейросети конкретно для нашего случая? Не пугаемся, но вот она:
$$
h(X) = h_W^{o}(h_W^{h}(X)) = g^{o}(g^{h}(X*W_{ih}) * W_{ho})
$$

Тут на самом деле нет ничего сложного, просто вложенность одного в дургое. По сути цепочка строится таким образом:
- Вычисляется произведение $XW$ для первого слоя (со своей матрцией весов);
- Применяется функция активации первого слоя $g(x)$;
- Полученный результат передается на второй слой как вход $X$;
- Продолжаем до конца слоев в нейросети.

То есть, никто не пишет обычно полную функцию активации - все понимают, что вычисление происходит последовательно от слоя к слою. Вот мы и разобрались с тем, как нейросеть вычисляет предсказания.

## Обучение нейросети

Обучение нейросети тоже происходит по алгоритму градиентного спуска, но никто не запрещает пользоваться другими алгоритмами оптимизации (
Алгоритм Левенберга — Марквардта, генетические алгоритмы, вариации алгоритма градиентного спуска и др.). Правда процесс обучения немного не классический. Функция предсказания у нас вычисляет результат от слоя к слою - такой процесс называется **прямое распространение (forward feed)**. Когда нейросеть вычислила первый раз результат во время обучения, мы можем сравнить результат с истинным значением и получить отклонения предсказания (ошибку). Тогда мы запускаем "некоторую *волну*", которая распространяет эту ошибку в обратном порядке от слоя к слою, чтобы на каждом слое обновить веса - это **обратное распространение (backpropagation)** ошибки.

<center>
<img src="https://docs.google.com/uc?export=download&id=1sdEgzwmkXEQfbDM2CjXXrveB-K30m2iD"/>
</center>


> Мы рассматривать типы нейросетей особо не будем - есть очень много классификаций на различных ресурсах, но обратите внимание на забавный факт! В рассматриваемой сети во время обучения (*train mode*) происходит два направления распространения информация, прямой ход и обратный. В режиме предсказания (*inference mode*) сеть работает полько применяя прямой ход информации. Поэтому такие сети называются *сети прямого распространения (feedforward networks)*. В качестве сравнения существуют рекуррентные сети, в которых информация в режиме предсказания распространяется не только прямым ходом - этакий режим памяти.

На всякий случай вспомним, что обновление веса по градиентному спуску происходит по формуле:
$$
w_{o_1} \leftarrow w_{o_1} - \alpha \frac{\partial J}{\partial w_{o_1}^1}
$$

Так как же делается это обратное распространение ошибки? Мы для этого выделим конкретную ветку и посмотрим, что и как расчитывается. 

Начнем с последнего слоя:

<center> 
<img src="https://docs.google.com/uc?export=download&id=1YloSxTLhIE_C2lUxW-uV7dzoVCGKRl9J"/>
</center>

Здесь принцип обучения не отличается от принципа обучения простого нейрона. Формулы все те же, для примера возьмем конкретный вес:
$$
\frac{\partial J}{\partial w_{o_1}^1} = 
\frac{\partial J}{\partial g_{o_1}} 
\frac{\partial g_{o_1}}{\partial z_{o_1}} 
\frac{\partial z_{o_1}}{\partial w_{o_1}^1} = 
(g(z_{o_1})-y)*g(z_{o_1})*(1-g(z_{o_1}))*a_{h_1}
$$

> $a_{h_1}$ - выход нейрона $h_1$ и, соответственно, вход для нейрона $o_1$

Обратите внимание, чтобы нам получить производную по весу последнего нейрона, нам нужно развернуть "цепочку" вычислений в обратную сторону по всем операциям.

В данном случае получить обновление не сложно, давайте перейдем к разбору того, а как добраться до весов в более глубоких слоях и попытаемся понять основную идею распространения!




Теперь рассмотрим такой кейс:

<center>
<img src="https://docs.google.com/uc?export=download&id=16A1u4RGgo6RIljet51UvjNodRfZxrLEl"/>
</center>

Здесь мы смотрим на конкретный вес, так как для всех остальных принцип будет тот же. Наша задача найти проихводную $\frac{\partial J}{\partial w_{h_1}^1}$, то есть, как надо поменять вес $w_{h_1}^1$, чтобы функцию потерь мы уменьшали.

Давайте для удобства запишем нынешний вариант формулы для вычисления:
$$
\hat{y} = g_{o_1}(z_{o_1}) \\
z_{o_1} = a_{h_1}*w_{o_1}^1 \\
a_{h_1} = g_{h_1}(z_{h_1}) \\
z_{h_1} = x_{1}*w_{h_1}^1
$$

Теперь мы можем полностью развернуть нашу прозводную, пройдя по двум нейронам (сначала разворачиваем скрытый нейрон, затем выходной):
$$
\frac{\partial J}{\partial w_{h_1}^1} = 
\frac{\partial J}{\partial g_{h_1}}
\frac{\partial g_{h_1}}{\partial z_{h_1}} 
\frac{\partial z_{h_1}}{\partial w_{h_1}^1} =
\frac{\partial J}{\partial g_{o_1}} 
\frac{\partial g_{o_1}}{\partial z_{o_1}} 
\frac{\partial z_{o_1}}{\partial g_{h_1}} 
\frac{\partial g_{h_1}}{\partial z_{h_1}} 
\frac{\partial z_{h_1}}{\partial w_{h_1}^1}
$$

А для выходного слоя была формула такая:
$$
\frac{\partial J}{\partial w_{o_1}^1} = 
\frac{\partial J}{\partial g_{o_1}} 
\frac{\partial g_{o_1}}{\partial z_{o_1}} 
\frac{\partial z_{o_1}}{\partial w_{o_1}^1}
$$


Давайте присмотримся и постараемся разобраться. При выводе мы всегда сталкиваемся с такой производной:
- Для $w_{o_1}$ мы сталкиваемся с $\frac{\partial J}{\partial g_{o_1}} 
\frac{\partial g_{o_1}}{\partial z_{o_1}}$;
- Для $w_{h_1}$ мы сталкиваемся с $\frac{\partial J}{\partial g_{h_1}} 
\frac{\partial g_{h_1}}{\partial z_{h_1}}$;

При этом эта часть всегда умножается на производную, которая в результате дает просто значение входа нейрона. А давайте запишем ее вот так:
$$
\delta_{o_1} = \frac{\partial J}{\partial g_{o_1}} \frac{\partial g_{o_1}}{\partial z_{o_1}} \\
\delta_{h_1} = \frac{\partial J}{\partial g_{h_1}} \frac{\partial g_{h_1}}{\partial z_{h_1}}
$$

Тогда получается, что для каждого нейрона уравнение производной получается следующим:
$$
\frac{\partial J}{\partial w_{o_1}^1} = \delta_{o_1}*\frac{\partial z_{o_1}}{\partial w_{o_1}^1} \\
\frac{\partial J}{\partial w_{h_1}^1} = \delta_{h_1}*\frac{\partial z_{h_1}}{\partial w_{h_1}^1}
$$

А так как $z$ у любого нейрона - это взвешенная сумма входов, то производная будет давать просто вход нейрона по этому весу (по которому берется производная)! То есть определение производной превратилось в нахождение дельты умноженной на вход нейрона. Давайте запомним дельту как **ошибка нейрона**.

> Уточним, под входом нейрона понимается значение, которое было подано на вход во время прямого прохода.

Так, уже выглядит проще! А теперь главный вопрос, как найти эту дельта? Здесь нам поможет длинный вывод производной для нейрона в скрытом слое:
$$
\frac{\partial J}{\partial g_{h_1}} 
\frac{\partial g_{h_1}}{\partial z_{h_1}} = 
\frac{\partial J}{\partial g_{o_1}} 
\frac{\partial g_{o_1}}{\partial z_{o_1}} 
\frac{\partial z_{o_1}}{\partial g_{h_1}}
\frac{\partial g_{h_1}}{\partial z_{h_1}} 
$$

или

$$
\delta_{h_1} = \delta_{o_1} * \frac{\partial z_{o_1}}{\partial g_{h_1}} * \frac{\partial g_{h_1}}{\partial z_{h_1}} 
$$

Что означает такая запись? Да именно то, что ошибку последующих нейронов (по глубине от выхода ко входу) мы можем рассчитать через ошибку предыдущих. Здесь в примере ошибка скрытого нейрона рассчитывается на основе ошибки выходного нейрона. И тут на секунду, ошибку выходного нейрона мы уже точно умеем рассчитывать! Круто! Осталось до конца разобраться с этой формулой!

Для начала, посмотрим на эту часть $\frac{\partial z_{o_1}}{\partial g_{h_1}}$ - это та часть, которая связывает взвешенную сумму $z_{o_1}$ нейрона и его вход (выход предыдущего нейрона $g_{h_1} = a_{h_1}$), то есть просто вес связи $w_{o_1}^1$!

Ну а выражение $\frac{\partial g_{h_1}}{\partial z_{h_1}}$ - просто производная функции активации предыдущего нейрона. Тоже ничего специфичного.

То есть, фактически, уравнение:
$$
\delta_{h_1} = \delta_{o_1} * \frac{\partial z_{o_1}}{\partial g_{h_1}} * \frac{\partial g_{h_1}}{\partial z_{h_1}}
$$

это получение ошибки следующего (при обратном распространении) слоя за счет умножения ошибки нейрона и производной функции активации! 

Такими шагами можно добраться до самых первых слоев даже для 100-слойной сети, но это уже совсем другая история про **затухание/взрыв градиента** и **влияние функции активации на распространение градиента**.

> Если вам интересно, что это за эффекты - обязательно прочтите про них!


## Для более интересующихся

Мы сейчас разбирались в терминах производных, но вы можете проверить себя и сформировать формулы производных для конкретных случаев. Например, для рассматриваемых производных были такие результаты:

$$
\frac{\partial J}{\partial g_{o_1}} = g(z_{o_1})-y \\
\frac{\partial g_{o_1}}{\partial z_{o_1}} = g(z_{o_1})*(1-g(z_{o_1}) \\
\frac{\partial z_{o_1}}{\partial g_{h_1}} = \frac{\partial z_{o_1}}{\partial a_{h_1}} = w_{o_1}^1 \\
\frac{\partial g_{h_1}}{\partial z_{h_1}} = g(z_{h_1})*(1-g(z_{h_1}) \\
\frac{\partial z_{h_1}}{\partial w_{h_1}^1} = x_1
$$

И сформируем полное уравнение:
$$
\frac{\partial J}{\partial w_{h_1}^1} = 
(g(z_{o_1})-y)*g(z_{o_1})*(1-g(z_{o_1}))*w_{o_1}^1 *
g(z_{h_1})*(1-g(z_{h_1})*x_1
$$

Для сравнения:
$$
\frac{\partial J}{\partial w_{o_1}^1} = 
(g(z_{o_1})-y)*g(z_{o_1})*(1-g(z_{o_1}))*a_{h_1}
$$

Но мы уже выяснили, что производные вычисляются через ошибки, поэтому полные формулы даже второго скрытого с конца слоя никто не хочет смотреть =)


На этом мы заканчиваем нашу чисто теоретическую часть, так как на сегодняшний день мы можем пользоваться благами готовых инструментов, понимая, как это все работает!

# Первые шаги в PyTorch

## Тензоры

Свое знакомство с нейросетями мы начнем с освоения фреймворка PyTorch. Тут нет ничего сложного, просто надо понять, что PyTorch привык работать со своим форматом массивов, поэтому мы встречаемся с первым понятием - **тензор**. По сути своей тензор ничем не отличается от массива, тоже может иметь любое количество размерностей, но PyTorch имеет свой класс для этого, поэтому надо привыкать работать с ним.

Давайте создадим тензор единиц размером $(3, 4)$, при этом мы изначально создадим массив через numpy и затем передадим его в функцию `torch.tensor()`:

In [ ]:
# TODO - создадим массив numpy размером (3, 4) и преобразуем в тензор
# NOTE - в помощь справка https://pytorch.org/docs/stable/generated/torch.tensor.html#torch.tensor

ones_numpy = None
ones_tensor = None

In [ ]:
# TEST

# Да, у тензора тоже есть атрибут .shape
assert np.all(ones_tensor.shape == np.array([3, 4]))

Можно было сразу создать тензор единиц с помощью `torch.ones()`, но тут важно понять, что numpy и torch могут тесно взаимодействовать и переводить данные из одного формата в другой! Давайте вернемся из тензора в формат numpy:

In [ ]:
# TODO - получите массив numpy с помощью метода тензора `torch.Tensor.numpy()`
data_tensor = torch.tensor([[3, 4], [1, 2]])
data_numpy = None

In [ ]:
# TEST
assert isinstance(data_numpy, (np.ndarray))

По сути своей тензоры идентичны матрицам, но есть две важные особенности:
- если присмотреться к аргументу `requires_grad`, то можно выяснить, что тензоры хранят не только значения, но и величины градиентов! Эти градиенты используются при обратном распространении ошибки системой `autograd` в PyTorch.
- также имеется аргумент `device`, который управляет тем, на каком вычислителе будут производиться операции. Да-да, вот так просто можно кинуть данные на видеокарту (если есть NVidia) и выполнить операции, но не думайте, что таким образом достигается лучшая производительность. Видеокарта умеет быстро выполнять огромные объемы простых вычислений, а кидать парочку массивов на сложение - не выгодно и будет дольше, чем на CPU!

В этом все отличия тензора, по всем операциям вы можете посмотреть [доки](https://pytorch.org/docs/stable/index.html), а мы двигаемся дальше!

## Модули

**Модули** в PyTorch - это классы, которые имеют определенный функционал обработки. Их еще можно назвать операциями. Например, есть модуль $MSE$, модуль сигмоиды, модуль Softmax и т.д.

В чем их отличие от простых функций или классов? При разработке нейросетей, например, на основе функции потерь вычисляются градиенты, которые дальше распространяются для обновления весов. Простые функции или операции нам только дадут конкретные значения, а использование модулей в PyTorch позволяет не только запомнить, как распространялись по сети вычисления, но и вычислить необходимые градиенты.

Другой важной особенностью является абстракция аппаратного кода. Мы не задумываемся, как производить вычисления на GPU или CPU, при этом в тензоре можно задать целевую аппаратуру для использования. Модули в этом плане хороши тем, что при выполнении операций также смотрят на аппаратуру переданных тензоров и выполняют операции на той аппаратуре, на которую нацелены тензоры (заданы через `device`). Таким образом, абстракция заключается в том, что мы не пишем отдельный код, а пользуемся одними и теми же модулями без каких-либо проблем.

Давайте теперь перейдем к знакомству и первым мы познакомимся с модулем линейного слоя. На деле это слой нейронов, но без функции активации:

In [ ]:
from torch import nn

# https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
neurons_layer = nn.Linear(
    in_features=3,  # Количество нейронов в предыдущем слое
                    #   или количество входов нейросети, если слой первый
    out_features=2  # Количество нейронов в слое (выходов слоя)
)

Как всегда, данный класс имеет атрибуты. Подробнее можем глянуть в доках, а пока посмотрим на веса:

In [ ]:
neurons_layer.weight

In [ ]:
neurons_layer.bias

Как видим, веса модуля тоже представлены тензором. При этом есть возможность отключить веса `bias`, чтобы слой имел только веса для входов, без смещения.

In [ ]:
# TODO - создайте слой нейронов и отключите bias

neurons_layer_no_bias = None

In [ ]:
# TEST
assert neurons_layer_no_bias.bias is None

Хорошо, мы смогли создать слой с `bias`, без него, но как им пользоваться? Очень просто! Модуль - это конкретная операция, поэтому все, что нам нужно - вызвать его!

In [ ]:
# Слой в три нейрона и двумя входами (данные или предыдущий слой)
layer = nn.Linear(3, 2)
# Зададим свои веса модели
layer.weight.data.fill_(1)
layer.bias.data.fill_(1)
# Создаем пример данных с одной записью и тремя признаками
input = torch.tensor([[1.5, 2, 3]])
print(f'Input: {input}')
print(f'Weights: {layer.weight}')
print(f'Bias: {layer.bias}')

result = layer(input)
print(f'\тResult: {result}')

manual_result = input @ layer.weight.T + layer.bias
print(f'Manual result: {manual_result}')

Шикарно, мы использовали модуль и затем проверили, что все работает корректно!

Смотрите, в тензоре еще есть атрибут `grad_fn`. На самом деле это как раз специфика модулей и применяемых тензоров. При использовании модуля он сразу вычисляет необходимые для градиента данные, а тензор может хранить информацию об этом градиенте, чтобы в дальнейшем использовать ее для распространения ошибки. Вот так красиво можно не волноваться о градиенте, а просто заниматься разработкой кода и анализом данных!

Пора бы нам уже обучить нашу первую нейросеть на PyTorch!

# Веса слоя нейросети - их первые значения

Инициализация весов - это вообще очень большой вопрос! Мы его здесь рассматривать не будем, но обратите внимание, что PyTorch решает эту проблему за вас! Проблема называется **проблема симметрии** и она очень сильно влияет на работу нейросети. Об этом с примером можно прочитать [здесь](https://www.quora.com/Why-dont-we-initialize-the-weights-of-a-neural-network-to-zero) и на многих других ресурсах! Именно поэтому при работе веса часто инициализируются случайными числами из некоторого распределения.

# Наша первая нейросеть

Сейчас мы создадим пачку точек (данных), чтобы научиться строить и работать с ними в PyTorch!

> Не забывайте, что реальная работа с данными и обучение моделей предполагает разделение на выборки:
- обучения - для тренировки моделей;
- валидация - для настройки гиперпараметров модели;
- тест - для окончательной оценки модели.

> В данной практике мы пропустим вопросы предобработки данных, тем не менее учитывайте, что использование модели в виде нейросети не сильно отличается в отношении подготовки данных и стандартизации!

In [ ]:
n_points = 100

real_W = [2, 0.7]
X_data = 4*np.sort(np.random.rand(n_points, 1), axis=0)+1
noize = 1*(np.random.rand(n_points, 1)-0.5)
y_data_true = real_W[0] + real_W[1]*X_data
y_data_noized = y_data_true + noize
y_data = y_data_noized[:, 0]

X_render = np.linspace(X_data[:, 0].min(), X_data[:, 0].max(), 100)
y_render = real_W[0] + real_W[1]*X_render

plt.scatter(X_data, y_data_noized, label='Данные')
plt.ylabel('$y$')
plt.xlabel('$x$')
plt.grid()
plt.legend()

Начнем как всегда с простого, получение предсказаний и оценка (визуальная и численная):

In [ ]:
# Мы будем добавлять строку задания seed при каждом создании слоя,
#   чтобы зафиксировать случайные числа при инициализации весов
torch.manual_seed(RANDOM_STATE)

# Создаем слой из одного нейрона в слое и одного входа
#   У слоя не будет активации (или можно назвать "линейной" активацией)
model = nn.Linear(1, 1)
# Переводим данные в формат тензора
#   Приводим к типу float методом .float()
X_tnsr = torch.tensor(X_data).float()
y_true_tnsr = torch.tensor(y_data).float()

# Исполняем слой - получаем предсказание
y_pred_tnsr = model(X_tnsr)

# Создаем модуль оценки loss методом MSE и оцениваем
loss_mod = nn.MSELoss()
loss_value = loss_mod(y_pred_tnsr, y_true_tnsr)

print(loss_value)

Глядите, результат оценки loss - это тоже тензор одиночного значения с информацией о градиенте! Если нам нужно использовать его с другими тензорами, то так и оставляем, а для того, чтобы просто получить скалярное значение, то можно воспользоваться методом `.item()`:

In [ ]:
loss_scalar = loss_value.item()
print(f'Scalar value: {loss_scalar}')

Отлично, эти методы полезны, когда мы работаем с тензорами! Теперь осталось посмотреть на наши данные:

In [ ]:
def plot_model(X, y_pred, y_true):
    plt.scatter(X, y_true, label='Данные')
    plt.plot(X, y_pred, 'k--', label='Предсказание модели')
    plt.ylabel('$Y$')
    plt.xlabel('$X$')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
# Переводим обратно в формат numpy
#   .detach() нужен, чтобы отсоединить информацию о градиенте
#   без этого будет ошибка - можете попробовать проверить
y_pred = y_pred_tnsr.detach().numpy()

plot_model(X_data, y_pred, y_data)

Как и ожидалось, случайные веса в нейросети не дают ожидаемого результата. Пора научиться обучать сеть!

> В PyTorch с фиксацией случайных чисел есть неопределенности, поэтому от запуска к запуску веса, а соответсвенно и предсказания, могут отличаться - не пугайтесь.

Что мы сейчас знаем:
- Как преобразовать данные в формат тензора
- Как создать однослойную нейронную сеть (пока что один нейрон в слое)
- Как оценить функцию потерь встроенными инструментами
- Как преобразовывать данные из тензора в специальных случаях (отсоединение градиента и получение скаляра)

> Если вы чего-то из этого списка не помните - лучше вернуться и повторить!

## Обучение нейросети

Хорошо, предсказания и оценка Loss получены, теперь остался вопрос, а как в PyTorch сделать обновление весов?

Для начала глянем на правило градиентного спуска:
$$
W := W - \alpha \frac{\partial J(W)}{\partial W}
$$

Таким образом делается это в несколько шагов:
- Нам нужно, чтобы все нейроны получили информацию о градиентах, делается это методом `.backward()` тензора `loss_value`, так как градиент рассчитывается на основе функции потерь;
- Нужно обновить веса в каждом нейроне, для этого:
    - Создадим объект алгоритма оптимизации, мы раньше использовали Gradient Descent, здесь мы используем его модификацию - Stochastic Gradient Descent (SGD). Хорошее описание различных методов оптимизации есть [здесь](https://ruder.io/optimizing-gradient-descent/).
    - Чтобы он знал, какие параметры оптимизировать, во время создания объекта оптимизатора передадим ему параметры модели;
    - После вычисления loss распространим градиент и выполним шаг оптимизатора;
    - Обнулим градиенты на слоях, потому что с каждым вызовом `.backward()` градиенты складываются.

Давайте проверим это, сделав один шаг обучения:

In [ ]:
# TODO - создайте модель нейросети из одного слоя и одного нейрона
#   подготовьте данные и получите предсказание
def create_model():
    torch.manual_seed(RANDOM_STATE)
    return None

def predict(model, X):
    return None

In [ ]:
# TEST
model = create_model()
# Создадим объект Stochastic Gradient Descent
optimizer = torch.optim.SGD(
    params=model.parameters(),  # Параметры модели
    lr=0.01                    # Коэффициент обучения
)
loss_op = nn.MSELoss()

X_tnsr = torch.tensor(X_data).float()
# Метод .view() - аналог np.reshape()
y_true_tnsr = torch.tensor(y_data).float().view((X_tnsr.shape[0], -1))

y_pred_tnsr = predict(model, X_tnsr)
loss_value = loss_op(y_pred_tnsr, y_true_tnsr)
print(loss_value)

# Обнуляем градиенты в модели
optimizer.zero_grad()
# Делаем распространение градиентов
loss_value.backward()
# Делаем шаг оптимизации - обновление весов
optimizer.step()

y_pred_tnsr_new = predict(model, X_tnsr)
loss_value_new = loss_op(y_pred_tnsr_new, y_true_tnsr)
print(loss_value_new)

In [ ]:
# Отобразим данные

y_pred_0 = y_pred_tnsr.detach().numpy()
y_pred_1 = y_pred_tnsr_new.detach().numpy()

plt.scatter(X_data, y_data)
plt.plot(X_data, y_pred_0, 'b--', label='Шаг 0')
plt.plot(X_data, y_pred_1, 'g--', label='Шаг 1')

plt.legend()
plt.grid()
plt.show()

Как видно по числам и графику - шаг оптимизации прошел успешно! Линия стала ближе к данным, а значит веса обновляются в верном направлении!


Теперь давайте представим нашу систему в виде графа вычислений:

![граф вычислений](https://docs.google.com/uc?export=download&id=1AMjPwq28O393LevN0E49cKUcqgby5J7-)

Здесь
* ребра (стрелки) - тензоры,
* узлы (прямоугольники) - модули.

По сути после каждой операции тензор-результат получает `grad_fn`, по которой можно обратно распространить ошибку и обновить параметры в модуле, если они есть.
> Например, в `nn.MSELoss()` нечего обновлять, но при этом он участвует в вычислениях и учитывается, если распространять градиент от `loss_value`.

Так мы разобрались, зачем тензору атрибут `grad_fn`. Теперь, что происходит, если на вход `nn.MSELoss()` мы подадим `y_pred_tnsr`, но перед этим сделаем `y_pred_tnsr = y_pred_tnsr.detach()`? 

А все не сложно: после вычисления, если мы распространим ошибку методом `.backward()`, то `nn.Linear()` не получит обновления параметров, так как `y_pred_tnsr` является связующим, а мы у него удалили информацию о градиенте!

Вот так в PyTorch происходит распространение ошибок и оптимизация параметров для минимизации скалярного тензора, от которого делается `.backward()`. Именно поэтому мы делаем обратное распространение от значения функции потерь!



## Продолжаем обучать нейросеть

Давайте теперь напишем функцию обучения, которой передадим модель, оптимизатор и на выходе хотим получить историю функции потерь

In [ ]:
# TODO - напишите функцию обучения модели 
#   с учетом переданных параметров и данных
# Мы не возвращаем модель, так как параметры обновляются прямо в ней
#   (в переданном снаружи объекте)
def fit_model(model, optim, loss_op, X, y, n_iter):
    loss_values = []

    return loss_values        

In [ ]:
# TEST
model = create_model()
# Создадим объект Stochastic Gradient Descent
optimizer = torch.optim.SGD(
    params=model.parameters(),
    lr=0.01
)
loss_op = nn.MSELoss()

loss_history = fit_model(
    model=model,
    optim=optimizer,
    loss_op=loss_op,
    X=X_data,
    y=y_data,
    n_iter=100
)

assert loss_history[-1] < 0.22

plt.plot(loss_history)
plt.grid()
plt.show()

### Задание - изучаем коэффициент обучения

Проверьте обучение модели при разных `lr` из списка `[0.1, 0.01, 0.001]` и отобразите графики обучения.

# Что такое batch и как мне предсказать свое значение?

> **Batch** - пакет данных. Нейросети по умолчанию работают батчами, то есть пакетами данных.

В нашем случае мы подавали на вход модели все имеющиеся данные, поэтому размер батча для нас - это количество записей в наборе. Формат батча обычно описывается определенными символами в зависимости от размерности и количества размерностей. 

Буквально сейчас мы учили сеть с форматом входного батча NM, где N - количество данных в батче (все данные, 100 записей), M - количество признаков в данных (в нашем случае 1). Выходной батч имел формат N1, то есть N предсказаний (по размеру входа) и 1 колонка с предсказанным значением.

По сути сейчас ничего особо нового, просто особенности терминологии, так как ранее мы тоже подавали на вход моделей 2D массивы. В случае работы с изображениями мы узнаем, что батч может иметь больше двух размерностей и не всегда данные подаются разом, так как вряд ли нам хватит памяти, чтобы загрузить миллион изображений в память.

Так вот, так как мы знаем, что вход нейросети предполагает определенный формат пакета, то для случая, если, например, мы захотим предсказать значение для некоторого нового вектора признаков (новая запись данных), то нам нужно привести к требуемому формату: 

In [ ]:
# Возьмем к примеру случай, когда у нас три признака в данных
x_new_sample = [1, 2, 3]
new_tnsr = torch.tensor(x_new_sample, dtype=torch.float)
print(new_tnsr.shape)
# Сейчас запись имеет формат M - кол-во признаков
# Чтобы привести его к формату NM, мы должны добавить 
#   еще одну размерность методом .unsqueeze()

# Передали аргумент 0, чтобы размерность добавилась в начале
new_batch = new_tnsr.unsqueeze(0)
print(new_batch.shape)

После добавления размерности, если данные уже предобработаны (стандартизированы, сгруппированы и т.д.), то можно подавать батч (хоть и из одной записи) на вход модели!

In [ ]:
x_new = [2]
in_data = torch.tensor(x_new, dtype=torch.float)
in_data = in_data.unsqueeze(0)

y_pred = model(in_data)
# Этим принтом увидим, что выход тоже батчем
print(y_pred.shape)

# Отсоединим градиент и переведем в формат numpy
y_pred = y_pred.detach().numpy()
# Так как батч имеет единственную запись - заберем данные из него
y_pred = y_pred[0]

plt.scatter(X_data, y_data)
plt.scatter(x_new, y_pred, marker='x', color='r')
plt.grid()
plt.show()

Видите красный крестик? Вот так модель предсказывает новые данные и при этом один (наш красный крестик) достаточно близок к данным.

Таким незамысловатым образом мы смогли получить предсказания по нашим новым данным и узнали, что такое батчи в нейросетях.

# Многослойные сети

На данный момент мы отлично справляемся с нейросетью, состоящей из одного нейрона! Настало время попробовать сделать многослойную сеть! Самый простой способ - сделать два модуля слоя и выполнить один за другим!

In [ ]:
torch.manual_seed(RANDOM_STATE)

# Делаем два нейрона в первом слое
layer1 = nn.Linear(1, 2)
# Так как в предыдущем два нейрона, то здесь два входа
layer2 = nn.Linear(2, 1)

# Данные примера
X_sample = torch.tensor([[1], [2], [3]]).float()

# Исполняем один за другим
l1_data = layer1(X_sample)
y_pred_tnsr = layer2(l1_data)

# Смотрим на предсказания
y_pred_tnsr

Первый способ объединения модулей в один является использвание [`nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html). Принцип работы с ним в том, что он объединяет операции в последовательность:

In [ ]:
seq_module = nn.Sequential(layer1, layer2)
print(seq_module)

y_pred_tnsr = seq_module(X_sample)
y_pred_tnsr


Другим способом является написание класса модели, который наследуется от `nn.Module`. Посмотрим, как это делается:

In [ ]:
class MyLinearModel(nn.Module):
    def __init__(self):
        # На этой строке вызывается конструктор класса, от которого наследуемся
        #   Она нужна, чтобы корректно настроить класс
        super().__init__()
        torch.manual_seed(RANDOM_STATE)

        self.layer1 = nn.Linear(1, 2)
        self.layer2 = nn.Linear(2, 1)
    
    # Метод, который нужно написать, чтобы вызов работал!
    def forward(self, X):
        l1_data = self.layer1(X)
        y_pred = self.layer2(l1_data)

        return y_pred

In [ ]:
model = MyLinearModel()
# При отображении показываются все слои внутри модели
print(model)

# Вот именно в этот момент происходит вызов метода forward() класса
y_pred_tnsr = model(X_sample)
y_pred_tnsr

Таким образом создается класс, который содержит всем необходимые слои (можно даже использовать `nn.Sequential` и другие вспомогательные классы внутри) и в нем пишется метод `forward()`, в котором прописываются действия со слоями. Потом объект этого класса можно просто вызывать и получать результат метода `forward()`! Отличная инкапсуляция!

## Задание - обучение многослойной нейронной сети

Самое время обучить модель и понять, лучше или хуже она работает, чем однослойная с одним нейроном:

In [ ]:
# TODO - обучите многослойную сеть и отобразите историю обучения 
#   и предсказания обученной модели
model = MyLinearModel()

Подумайте над идеей того, что модель теперь состоит из двух слоев и суммарно трех нейронов, но при этом характер предсказания (прямая линия) не изменился. Таким образом, нейронная сеть из слоев с линейной активацией не даст ничего, кроме линейной модели!

# Как оценить работу нейросети?

Нейросеть = еще один вид модели машинного обучения. При том, что мы решали задачу регрессии, то и все метрики, которые мы использовали для моделей регрессии, применимы здесь!

Аналогично, сейчас рассмотрим задачу классификации, но и там такие же принципы для оценки и метрики.

In [ ]:
# TODO - напишите функцию оценки работы модели по метрике R2 
#   (не забудьте импорт нужной функции из sklearn)

def evaluate_r2(model, X, y):
    return r2_value

In [ ]:
from sklearn.metrics import r2_score

r2_value = evaluate_r2(model, X_data, y_data)
print(f'R2: {r2_value}')

assert np.isclose(r2_value, 0.79912)

# Я выбираю нелинейность!

Для работы с данными, в которых нам нужно определить линейные зависимости, достаточно нейросети без активации (или просто нейрона). Теперь, во-первых, давайте начнем работать с данными, которые имеют явную нелинейность, а во-вторых, посмотрим, как добавить слою функцию активации, чтобы добавить нейросети свойство нелинейности.

Вот так будут выглядеть наши данные:

In [ ]:
X_data = np.linspace(2, 10, 100)[:, None]
y_data = 1/X_data[:,0]*5 + np.random.normal(size=X_data.shape[0])/7 + 2
# y_data = (-1)*X_data[:,0]**2+(10)*X_data[:,0] + np.random.normal(size=X_data.shape[0]) + 5

# Посмотрим на данные
plt.scatter(X_data[:,0], y_data)
plt.grid(True)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

Отлично, линейная модель тут уже вряд ли справится, нам нужно научиться добавлять слоям нелинейную активацию!

Давайте начнем с одного слоя и сделаем ему функцию активации:

In [ ]:
# Создаем слой и задаем свой вес
layer = nn.Linear(1, 1, bias=False)
layer.weight.data.fill_(10)
# Создаем модуль сигмоиды
activation_func = nn.Sigmoid()

# Данные для примера
X_sample = torch.tensor([[-10], [0], [10]]).float()
print(f'Input: {X_sample}')

# Исполняем вычисления результатов слоя
layer_result = layer(X_sample)
# Исполняем модуль сигмоиды
act_result = activation_func(layer_result)

print(f'Layer result: {layer_result}')
print(f'Activation result: {act_result}')

Вот таким несложным образом можно добавить в нейросеть нелинейность. Можно создать модуль и исполнять его.

Другим более предпочтительным способом является не создание модуля, а просто исполнение функции сигмоиды:

In [ ]:
layer_result = layer(X_sample)
y_pred = torch.sigmoid(layer_result)

print(f'Input: {X_sample}')
print(f'Layer result: {layer_result}')
print(f'Activation result: {act_result}')

Мы видим аналогичный результат как по значениям, так и по функции `grad_fn`. То есть при написании класса модели можно прямо в методе `forward()` вызывать функцию сигмоиды (или другой функции активации). 

## Задание - нелинейная сеть

Реализуйте код двуслойной сети по архитектуре `[2, 1]`:
- 2 нейрона в скрытом слое;
- 1 нейрон в выходной слое.

Скрытый слой должен иметь сигмоиду в качестве активации, выходной слой должен иметь линейную активацию (без активации). После написания обучите модель и посмотрите на предсказания модели:

In [ ]:
# TODO - реализуйте модель нейронной сети с нелинейностью, 
#   обучите и оцените модель

class NonlinearNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(RANDOM_STATE)
        # И здесь надо создать слои

    def forward(self, X):
        return y_pred

In [ ]:
model = NonlinearNeuralNetwork()
optimizer = torch.optim.SGD(
    params=model.parameters(),
    lr=0.1
)
loss_op = nn.MSELoss()

loss_history = fit_model(
    model=model,
    optim=optimizer,
    loss_op=loss_op,
    X=X_data,
    y=y_data,
    n_iter=1000
)

plt.plot(loss_history)
plt.grid()
plt.show()

X_tnsr = torch.tensor(X_data).float()
y_pred_tnsr = model(X_tnsr)
y_pred = y_pred_tnsr.detach().numpy()

plot_model(X_data, y_pred, y_data)

Отлично! По результатам обучения видно, что модель с нелинейностью может иметь нелинейный характер и описывать нелинейные зависимости. Можете самостоятельно оценить работу по численным показателям и кроссвалидацией постараться сделать модель еще точнее!

# Нейросеть для классификации

Думаю, и так понятно, что нейросеть не ограничивается только задачей регрессии, поэтому мы зацепим еще и работу модели для задачи классификации! Создадим данные для задачи классификации:

In [ ]:
from sklearn.datasets import make_classification, make_moons

X_data, y_data = make_moons(
    n_samples=1000,
    noise=.1,
    random_state=RANDOM_STATE
)

pnts_scatter = plt.scatter(X_data[:, 0], X_data[:, 1], marker='o', c=y_data, s=50, edgecolor='k')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.grid(True)
plt.legend(handles=pnts_scatter.legend_elements()[0], labels=['0', '1'])

Отлично! Данные есть, теперь можно переходить к написанию модели и обучению!

In [ ]:
class ClassificationNN(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(RANDOM_STATE)
        self.layer1 = nn.Linear(2, 2)
        self.layer2 = nn.Linear(2, 1)
    
    def forward(self, x):
        out = torch.sigmoid(self.layer1(x))
        out = self.layer2(out)
        y_prob = torch.sigmoid(out)
        return y_prob

Небольшая модель сделана! Обратите внимание, что у нас выход - это уже не чистое значение линейного слоя, а функция сигмоиды! Так как задача состоит в бинарной классификации, то нам достаточно сказать степень уверенности и задаться порогом, как обычно.

Теперь напишем цикл обучения, чтобы обучить модель, при этом для классификации нам нужна другая функция потерь - воспользуемся [`nn.BCELoss()`](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html):

In [ ]:
# TODO - напишите цикл обучения модели
# - Создайте модель
# - Задайте оптимизатор (SGD)
# - Создайте модуль функции потерь
# - Запустите обучение через fit_model() - видали, мы даже функцию не переписывали!
# - Отобразите историю обучения

In [ ]:
from sklearn.metrics import classification_report

def show_classification_report(model, X, y):
    X_tnsr = torch.tensor(X).float()
    y_pred_prob = model(X_tnsr).detach().numpy()

    y_pred = y_pred_prob > 0.5

    rep = classification_report(y, y_pred)
    print(rep)

show_classification_report(model, X_data, y_data)

Теперь еще напишем метод визуализации данных, чтобы посмотреть на пространство принятия решений:

In [ ]:
def plot_2d_decision_boundary(model, X, y):
    x1_vals = np.linspace(X[:,0].min()-0.5, X[:,0].max()+0.5, 100)
    x2_vals = np.linspace(X[:,1].min()-0.5, X[:,1].max()+0.5, 100)
    xx1, xx2 = np.meshgrid(x1_vals, x2_vals)

    X_tnsr = torch.tensor(np.c_[xx1.ravel(), xx2.ravel()]).float()
    y_pred = model(X_tnsr).detach()
    y_pred = y_pred.reshape(xx1.shape)

    plt.contourf(xx1, xx2, y_pred)
    pnts_scatter = plt.scatter(X[:, 0], X[:, 1], c=y, s=30, edgecolor='k')
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.grid(True)
    plt.legend(handles=pnts_scatter.legend_elements()[0], labels=['0', '1'])
    plt.show()

plot_2d_decision_boundary(model, X_data, y_data)

Хммм, явный клиничейский случай недообучения! Модель не может разделить столь нелинейные данные - давайте добавим модели сложности: три слоя и больше нейронов в слое!

In [ ]:
class ClassificationNNv2(nn.Module):
    def __init__(self):
        super().__init__()
        HIDDEN = 20

        torch.manual_seed(RANDOM_STATE)
        self.layers = nn.Sequential(
            nn.Linear(2, HIDDEN),
            nn.Sigmoid(),
            nn.Linear(HIDDEN, HIDDEN),
            nn.Sigmoid(),
            nn.Linear(HIDDEN, 1),
        )
    
    def forward(self, x):
        return torch.sigmoid(self.layers(x))

In [ ]:
# TODO - обучите модель снова и посмотрите на показатели (убедитесь, что лучше не стало...)

Теперь давайте попробуем поменять функции активации слоев (сигмоида выхода - не функция активации слоя!) с сигмоиды на гиперболический тангенс (в английском назван tanh). Найдите в документации PyTorch соответствующую функцию и обучите третью версию модели. Оцените, насколько изменился характер предсказания модели!

In [ ]:
# TODO - замените функцию активации слоев и обучите модель, сделайте выводы
class ClassificationNNv3(nn.Module):
    

    def forward(self, x):
        return torch.sigmoid(self.layers(x))

# Как сохранить модель?

Один из актуальных вопросов - а как мне сохранить модель, чтобы сохранить результаты обучения? Мне же не придется обучать модель каждый раз заново? Ответ, конечно нет! И для этого PyTorch имеет очень простой функционал!

In [ ]:
# Задаем путь, по которому хотим сохранить параметры модели
SAVE_PATH = 'my_model.pth'
# Вызываем функцию сохранения
# Сохраняем параметры модели!
torch.save(model.state_dict(), SAVE_PATH)

После сохранения в файловой системе должен появиться файл с названием модели! Вот так можно в файл перенести параметры. А как загрузить их?

In [ ]:
loaded_state_dict = torch.load(SAVE_PATH)

model = ClassificationNN()
model.load_state_dict(loaded_state_dict)

plot_2d_decision_boundary(model, X_data, y_data)

Отлично! Мы создали новую модель, загрузили сохраненные параметры и все работает! В этом подходе нужно учитывать следующую особенность, если поменяется архитектура модели, то параметры не смогут загрузиться. В остальном можно таким образом переносить обученную модель и использовать ее где угодно (где есть код, чтобы создать объект модели).

# Результаты

Мы рассмотрели очень серьезную тему, нейронные сети и применение фреймворка PyTorch! Тем не менее были пропущены такие вещи, как регуляризация нейросетей с помощью `nn.Dropout`, необходимость предобработки данных перед подачей на вход нейросети и другие.

Сам по себе фреймворк очень мощный, поэтому мы многое узнаем из новых практик, но главное - вы все можете узнать и попробовать сами! Главное, не бойтесь пробовать и испытывать!

# Выводы - задание

Вопрооооосики!

1. Из каких частей состоит нейрон? 
2. Может ли нейрон быть без функции активации? Что из этого получится (какой характер сети)? 
3. Можно ли в качестве функции активации выбрать кусочно-линейную функцию с разрывами?
4. Почему лучше не использовать функцию параболы в качестве функции активации? 
5. Какие слои нейросети можно назвать "скрытыми"? 
6. Сколько скрытых слоёв может быть в нейросети?
7. Можно ли использовать разные функции активации для нейронов в одном слое? 
8. Как называется процесс, когда функция предсказания работает от слоя к слою? 
9. Какое процесс описывает получение ошибки предсказания? 
10. Почему изначально значения весов устанавливаются случайным образом? 
11. Зачем нужна выборка-валидация? В чём отличие от выборки-теста? 
12. В чём разница между стохастическим градиентным спуском и полным градиентным спуском?
13. Что такое регуляризация? Зачем она нужна? 